In [17]:
import os

from network_wrangler import RoadwayNetwork, TransitNetwork, ProjectCard, Scenario

%config IPCompleter.greedy=True

import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define Base Files, Networks, and Scenario

In [19]:
STPAUL_DIR = os.path.join(os.getcwd(),'../','example','stpaul')
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR,"shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR,"link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR,"node.geojson")



In [6]:
road_net = RoadwayNetwork.read(
    link_file= STPAUL_LINK_FILE, 
    node_file=STPAUL_NODE_FILE, 
    shape_file=STPAUL_SHAPE_FILE, 
    fast=True
)

In [10]:
transit_net = TransitNetwork.read(STPAUL_DIR)

In [35]:
base_scenario = {
    "road_net": road_net,
    "transit_net": transit_net,
}

## Define and Create "No Build" Network

In this case, we will create a scenario based on all projects in a directory with a certain filename pattern

In [34]:
my_scenario_nobuild = Scenario.create_scenario(
    base_scenario=base_scenario, 
    card_directory = os.path.join(STPAUL_DIR, "project_cards"),
    glob_search = "*attribute*.yml"
)

In [38]:
my_scenario_nobuild.get_project_names()

['Improve Express Bus Frequency',
 '6th St E Road Diet',
 'Blue Line Service Enhancement',
 'Blue Line Service Enhancement',
 '6th Street Transitway']

In [40]:
my_scenario_nobuild.apply_all_projects()

In [41]:
my_scenario_nobuild.applied_projects

[]

## Specify specific cards to add for a build scenario

This scenario will be tiered off of the no-build scenario

In [42]:
BUILD_CARD_FILENAMES = [
        "4_simple_managed_lane.yml",
    ]

project_cards_list = [
    ProjectCard.read(os.path.join(STPAUL_DIR, "project_cards", filename), validate=False)
    for filename in BUILD_CARD_FILENAMES
]

In [44]:
my_scenario_build_alt1 = Scenario.create_scenario(
    base_scenario=my_scenario_nobuild.__dict__, 
    project_cards_list=project_cards_list
)

my_scenario_build_alt1.applied_projects

AttributeError: 'Scenario' object has no attribute 'get'

In [ ]:
my_scenario_build_alt1.apply_all_projects()

In [ ]:
my_scenario_build_alt1.applied_projects